In [1]:
import polars as pl
import pandas as pd
import time
from pathlib import Path

In [9]:
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

data_path = project_root / "data" / "raw"
bronze_path = project_root / "data" / "bronze"

bronze_path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data path: {data_path}")
print(f"Bronze folder created: {bronze_path}")

Project root: c:\python-projects\F1-pipeline
Data path: c:\python-projects\F1-pipeline\data\raw
Bronze folder created: c:\python-projects\F1-pipeline\data\bronze


In [11]:
csv_files = list(data_path.glob("*.csv"))
print(f"Number of CSV files: {len(csv_files)}\n")
for file in csv_files:
    print(f"  - {file.name}")

Number of CSV files: 14

  - circuits.csv
  - constructors.csv
  - constructor_results.csv
  - constructor_standings.csv
  - drivers.csv
  - driver_standings.csv
  - lap_times.csv
  - pit_stops.csv
  - qualifying.csv
  - races.csv
  - results.csv
  - seasons.csv
  - sprint_results.csv
  - status.csv
